# Imports 

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
plt.style.use('ggplot') #https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html
%matplotlib inline

D:\anaconda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\anaconda\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
D:\anaconda\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Get the data

In [2]:
unis_1819 = pd.read_pickle("data/unis_1819.pkl")
unis_2122 = pd.read_pickle("data/unis_2122.pkl")
summary_1819 = pd.read_pickle("data/summary_1819.pkl")
summary_2122 = pd.read_pickle("data/summary_2122.pkl")
course_1819 = pd.read_pickle("data/course_1819.pkl")
course_2122 = pd.read_pickle("data/course_2122.pkl")

# Pre-process